1. Examine the first few rows of `ad_clicks`.

In [1]:
import pandas as pd

ad_clicks = pd.read_csv('ad_clicks.csv')
print(ad_clicks.head())

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google   2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook    7 - Sunday   

  ad_click_timestamp experimental_group  
0               7:18                  A  
1                NaN                  B  
2                NaN                  A  
3                NaN                  B  
4                NaN                  B  


2. Your manager wants to know which ad platform is getting you the most views.

How many views (i.e., rows of the table) came from each `utm_source`?

In [2]:
utm_source = ad_clicks.groupby('utm_source').user_id.count().reset_index()
print(utm_source)

  utm_source  user_id
0      email      255
1   facebook      504
2     google      680
3    twitter      215


3. If the column `ad_click_timestamp` is not null, then someone actually clicked on the ad that was displayed.

Create a new column called `is_click`, which is True if `ad_click_timestamp` is not null and False otherwise.

In [3]:
ad_clicks['is_click'] = ad_clicks['ad_click_timestamp'].notnull()
print(ad_clicks.head(10))

                                user_id utm_source            day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google   6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook     7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter    2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google    2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook     7 - Sunday   
5  013b0072-7b72-40e7-b698-98b4d0c9967f   facebook     1 - Monday   
6  0153d85b-7660-4c39-92eb-1e1acd023280     google   4 - Thursday   
7  01555297-d6e6-49ae-aeba-1b196fdbb09f     google  3 - Wednesday   
8  018cea61-19ea-4119-895b-1a4309ccb148      email     1 - Monday   
9  01a210c3-fde0-4e6f-8efd-4f0e38730ae6      email    2 - Tuesday   

  ad_click_timestamp experimental_group  is_click  
0               7:18                  A      True  
1                NaN                  B     False  
2                NaN                  A     False  
3                NaN            

4. We want to know the percent of people who clicked on ads from each `utm_source`.

Start by grouping by `utm_source` and `is_click` and counting the number of `user_id`‘s in each of those groups. Save your answer to the variable `clicks_by_source`.

In [4]:
clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()

5. Now let’s pivot the data so that the columns are `is_click` (either `True` or `False`), the index is `utm_source`, and the values are `user_id`.

Save your results to the variable `clicks_pivot`.

In [5]:
clicks_pivot = clicks_by_source.pivot(
  index='utm_source',
  columns='is_click',
  values='user_id'
).reset_index()
print(clicks_pivot)

is_click utm_source  False  True
0             email    175    80
1          facebook    324   180
2            google    441   239
3           twitter    149    66


6. Create a new column in `clicks_pivot` called `percent_clicked` which is equal to the percent of users who clicked on the ad from each `utm_source`.

Was there a difference in click rates for each source?

In [6]:
clicks_pivot['percent_clicked'] = clicks_pivot[True] / (clicks_pivot[True] + clicks_pivot[False]) * 100.0

print(clicks_pivot)

is_click utm_source  False  True  percent_clicked
0             email    175    80        31.372549
1          facebook    324   180        35.714286
2            google    441   239        35.147059
3           twitter    149    66        30.697674


## Analyzing an A/B Test

7. The column `experimental_group` tells us whether the user was shown Ad A or Ad B.

Were approximately the same number of people shown both adds?

In [7]:
experimental_group = ad_clicks.groupby('experimental_group').user_id.count().reset_index()
print(experimental_group)

  experimental_group  user_id
0                  A      827
1                  B      827


8. Using the column `is_click` that we defined earlier, check to see if a greater percentage of users clicked on Ad A or Ad B.

In [8]:
percentage_A_B = ad_clicks.groupby(['experimental_group', 'is_click']).user_id.count().reset_index().pivot(
  index='experimental_group',
  columns='is_click',
  values='user_id'
).reset_index()
print(percentage_A_B)

is_click experimental_group  False  True
0                         A    517   310
1                         B    572   255


9. The Product Manager for the A/B test thinks that the clicks might have changed by day of the week.

Start by creating two DataFrames: `a_clicks` and `b_clicks`, which contain only the results for A group and B group, respectively.

In [9]:
a_clicks = ad_clicks[ad_clicks['experimental_group'] == 'A']
b_clicks = ad_clicks[ad_clicks['experimental_group'] == 'B']

10. For each group (`a_clicks` and `b_clicks`), calculate the percent of users who clicked on the ad by `day`.

In [10]:
a_clicks_by_day = a_clicks.groupby(['day', 'is_click']).user_id.count().reset_index().pivot(
  index='day',
  columns='is_click',
  values='user_id'
).reset_index()

b_clicks_by_day = b_clicks.groupby(['day', 'is_click']).user_id.count().reset_index().pivot(
  index='day',
  columns='is_click',
  values='user_id'
).reset_index()

a_clicks_by_day['percentage'] = a_clicks_by_day[True] / (a_clicks_by_day[True] + a_clicks_by_day[False]) * 100.0

b_clicks_by_day['percentage'] = b_clicks_by_day[True] / (b_clicks_by_day[True] + b_clicks_by_day[False]) * 100.0

print(a_clicks_by_day)
print(b_clicks_by_day)


is_click            day  False  True  percentage
0            1 - Monday     70    43   38.053097
1           2 - Tuesday     76    43   36.134454
2         3 - Wednesday     86    38   30.645161
3          4 - Thursday     69    47   40.517241
4            5 - Friday     77    51   39.843750
5          6 - Saturday     73    45   38.135593
6            7 - Sunday     66    43   39.449541
is_click            day  False  True  percentage
0            1 - Monday     81    32   28.318584
1           2 - Tuesday     74    45   37.815126
2         3 - Wednesday     89    35   28.225806
3          4 - Thursday     87    29   25.000000
4            5 - Friday     90    38   29.687500
5          6 - Saturday     76    42   35.593220
6            7 - Sunday     75    34   31.192661


11. Compare the results for `A` and `B`. What happened over the course of the week?

Do you recommend that your company use Ad A or Ad B?

In [11]:
print("Based on last week's data analysis, Ad A has more clicks than Ad B. Therefore, I recommend that our company should use Ad A.")

Based on last week's data analysis, Ad A has more clicks than Ad B. Therefore, I recommend that our company should use Ad A.
